IMPORTS AND CONSTANTS

In [ ]:
from xml.dom.expatbuilder import parseFragmentString
import pyxdf
import pandas as pd
from moviepy.editor import VideoFileClip
import numpy as np
from matplotlib import pyplot as plt
import mne

%matplotlib qt

bands = {'delta': [0.5, 4], 'tehta': [4, 8], 'alpha': [8, 13], \
         'beta': [13, 30], 'gamma': [30, 50]}
xdfFilePath = 'xdfs\pesa.xdf'
csvFilePath = 'csvs\pesa.csv'

plt.figure()
plt.close()

FETCHING VIDEO DURATIONS:

In [ ]:
videopath='C:\\Users\\Duca\\OneDrive - student.etf.bg.ac.rs\\ETF\\SESTI SEMESTAR\\AES\\PROJEKAT\\VIDEO\\VIDEOS\\'

video_names = pd.read_csv(csvFilePath, usecols=['Stimulus'])
vidind = [name[0:-4] for name in video_names['Stimulus']]
viddur = {}
for name in vidind:
    viddur[name] = VideoFileClip(videopath + name + '.mp4').duration
vidind

LOADING THE DATA:

In [ ]:
data, header = pyxdf.load_xdf(xdfFilePath)
signal = data[0]['time_series']
timestamps = data[0]['time_stamps']
ch_n = int(data[0]['info']['channel_count'][0])

chnames = []
for i in range(ch_n):
    chnames.append(data[0]['info']['desc'][0]['channels'][0]['channel'][i]['label'][0])
print(chnames)
print("Data loaded.")

sfreq = int(float(data[0]["info"]["nominal_srate"][0]))
info = mne.create_info(chnames, sfreq, "eeg")
raw = mne.io.RawArray(signal.T*1e-3/50/2, info) # CHECK THIS SCALING!!!
#raw.crop(0, raw.times[-1]-30.)
raw.set_montage('standard_1020', match_alias=True)
#* (1e-3 / 50 / 2)

segments = []

In [ ]:
raw.plot(scalings=dict(eeg=1e-4), duration=1, start=14, block=True)

In [ ]:
raw.plot_psd(fmax=125, average=True)

In [ ]:
filtered_raw = raw.copy().filter(l_freq=1., h_freq=40.).notch_filter(freqs=[50])
ica = mne.preprocessing.ICA(n_components=0.99, max_iter='auto', random_state=93)

ica.fit(filtered_raw)

In [ ]:
ica.plot_sources(raw, block=True, start=20., stop=40.)
ica.plot_components()

Manually selecting potentially bad channels, as per above generated graphs:

In [ ]:
bad_eog = [0, 3, 13]
bad_emg = [7, 10, 12]
bad_ecg = [4]
bads = bad_eog + bad_emg + bad_ecg

We need to check the potentially bad channels manually:

In [ ]:
ica.plot_properties(raw, picks=bads)

(https://labeling.ucsd.edu/tutorial/labels)
After checking, it seems that only some were indeed bad:

In [ ]:
ica.exclude = [0, 3, 4]

0 is vertical eye movement artifact, 3 is horizontal, and 4 is ECG artifact. We now apply ICA:

In [ ]:
raw_ica = filtered_raw.copy()
ica.apply(raw_ica)

SEGMENTING INTO SMALLER RAW FILES

In [ ]:
segments = []

current_t = raw._last_time
current_t-= 30
vidind.reverse()
for key in vidind:
    currentDur = viddur[key]
    current_t  -= currentDur
    segments.append({'raw': raw_ica.copy().crop(current_t, current_t+currentDur), 'ID': key})
    current_t -= 15
segments

CALCULATING THE FEATURES OF SIGNALS
-------------------------------------------------------------------------------------------------------------------------------

CALCULATING FREQUENCY CHARACTERISTICS OF SIGNAL PER SEGMENT

In [ ]:
for segment in segments:
    psds, freqs = mne.time_frequency.psd_welch(segment['raw'])
    segment['psds'] = psds
    segment['freqs'] = freqs
    segment['psds_avg'] = np.average(segment['psds'], axis=0)

In [ ]:
for segment in segments:
    avg_power = np.average(segment['psds_avg'])
    print(avg_power)
    segment['avg_pow']    = avg_power*1e11
    segment['peak_mag']   = np.max(segment['psds_avg'])/avg_power
    segment['bottom_mag'] = np.min(segment['psds_avg'])/avg_power
    segment['peak_freq']  = segment['freqs'][np.argmax(segment['psds_avg'])]
    segment['std']        = np.std(segment['psds_avg'])*1e11
    for bandname in bands.keys():
        segment[bandname] = {}
        band_indices = [i for i in range(len(segment['freqs']))
                            if segment['freqs'][i]>=bands[bandname][0] and segment['freqs'][i]<=bands[bandname][1]]
        banded_signal = np.array(segment['psds_avg'])[band_indices]
        segment[bandname]['average']    = np.average(banded_signal)/avg_power
        segment[bandname]['peak_mag']   = np.max(banded_signal)/avg_power
        segment[bandname]['bottom_mag'] = np.min(banded_signal)/avg_power
        segment[bandname]['peak_freq']  = segment['freqs'][np.argmax(banded_signal)+band_indices[0]]
        segment[bandname]['std']        = np.std(banded_signal/avg_power)

DETERMINING VIDEO EMOTIONS
------

In [ ]:
def determine_emotion(valence, arousal) -> str:
    if valence<=5:
        v = 1
    else:
        v = 0

    if arousal > 5:
        a = 0
    else:
        a = 1

    return [v, a]

In [ ]:
v_grades = pd.read_csv(csvFilePath)
print(v_grades)

for segment in segments:
    id = str(segment['ID'])
    vidname = id+'.mp4'
    
    valence = v_grades.loc[v_grades['Stimulus']==vidname]['Valence'].iloc[0]
    arousal = v_grades.loc[v_grades['Stimulus']==vidname]['Arousal'].iloc[0]

    emo = determine_emotion(valence, arousal)
    segment['PLES'] = emo[0]
    segment['AROS'] = emo[1]

SAVING DATA
----

First, we need column names:

In [ ]:
framecols = []
framecols.append('ID')
framecols.append('AVG')
framecols.append('PEAK')
framecols.append('BOTTOM')
framecols.append('PEAKF')
framecols.append('STD')
for key in bands.keys():
    framecols.append(key + '_AVG')
    framecols.append(key + '_PEAK')
    framecols.append(key + '_BOTTOM')
    framecols.append(key + '_PEAKF')
    framecols.append(key + '_STD')
framecols.append('VALENCE')
framecols.append('AROUSAL')

print(framecols)


In [ ]:
frame_to_save = pd.DataFrame()

ind = 0
for segment in segments:
    featurelist = []
    featurelist.append(segment['ID'])
    featurelist.append(segment['avg_pow'])
    featurelist.append(segment['peak_mag'])
    featurelist.append(segment['bottom_mag'])
    featurelist.append(segment['peak_freq'])
    featurelist.append(segment['std'])
    for bandname in bands:
        featurelist.append(segment[bandname]['average'])
        featurelist.append(segment[bandname]['peak_mag'])
        featurelist.append(segment[bandname]['bottom_mag'])
        featurelist.append(segment[bandname]['peak_freq'])
        featurelist.append(segment[bandname]['std'])
    featurelist.append(segment['PLES'])
    featurelist.append(segment['AROS'])
    frame_to_save[ind] = featurelist
    ind += 1
    
frame_to_save.set_axis(framecols, axis='index', inplace=True)
frame_to_save = frame_to_save.T
frame_to_save.to_csv('Signal_features.csv')